In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./text_to_number.csv')
df.tail()

,number,text
995,995,nine hundred and ninety-five
996,996,nine hundred and ninety-six
997,997,nine hundred and ninety-seven
998,998,nine hundred and ninety-eight
999,999,nine hundred and ninety-nine


In [2]:
# Vectorize the data.
input_texts_ = []
input_characters = set()

for line in df['text']:
    for input_text in line:
        if len(input_text)>0:
            input_texts_.append(input_text)
            for char in input_text:
                if char not in input_characters:
                    input_characters.add(char)

input_token_index = dict(
    [(char, i+1) for i, char in enumerate(input_characters)])

input_token_index['~']=0

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())


In [3]:
input_characters = sorted(list(input_characters))
num_tokens = len(input_characters)
max_seq_length = max([len(txt) for txt in df['text']])

print('Number of samples:', len(df['text']))
print('Number of unique input tokens:', num_tokens)
print('Max sequence length for inputs:', max_seq_length)

input_data = np.zeros(
    (len(df['text']), max_seq_length, num_tokens+1),
    dtype='int32')

for i, input_text in enumerate(df['text']):
    for t, char in enumerate(input_text):
        input_data[i, t, input_token_index[char]] = 1

Number of samples: 1000
Number of unique input tokens: 21
Max sequence length for inputs: 31


In [4]:
base_idx = [x for x in range(21)]
random_idx = [x for x in np.random.choice(1000, 150, replace=False)]
random_idx.extend(base_idx)
random_idx = list(np.unique(random_idx))
non_idx = [x for x in list(np.arange(1000)) if x not in random_idx]
train_x = input_data[random_idx]
train_y = df['number'][random_idx]
valid_x = input_data[non_idx]
valid_y = df['number'][non_idx]

In [70]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import RNN, concatenate
from keras import *

class LSTM2Cell(Layer):
    """Cell class for the LSTM layer.
    # Arguments
        units: Positive integer, dimensionality of the output space.
        activation: Activation function to use
            (see [activations](../activations.md)).
            Default: hyperbolic tangent (`tanh`).
            If you pass `None`, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        recurrent_activation: Activation function to use
            for the recurrent step
            (see [activations](../activations.md)).
            Default: hard sigmoid (`hard_sigmoid`).
            If you pass `None`, no activation is applied
            (ie. "linear" activation: `a(x) = x`).x
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al. (2015)](
            http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf).
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, either 1 or 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
    """

    def __init__(self, units,
                 activation='relu',
                 recurrent_activation='linear',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 NALU_initializer='zeros',
                 bias_initializer='zeros',
                 unit_forget_bias=True,
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 NALU_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 NALU_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=1,
                 **kwargs):
        super(LSTM2Cell, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.recurrent_activation = activations.get(recurrent_activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.NALU_initializer = initializers.Orthogonal(gain=0.0001, seed=123)
        self.bias_initializer = initializers.get(bias_initializer)
        self.unit_forget_bias = unit_forget_bias

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
        self.NALU_regularizer = regularizers.get(NALU_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(recurrent_constraint)
        self.NALU_constraint = constraints.get(NALU_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = min(1., max(0., dropout))
        self.recurrent_dropout = min(1., max(0., recurrent_dropout))
        self.implementation = implementation
        self.state_size = (self.units, self.units)
        self.output_size = self.units
        self._dropout_mask = None
        self._recurrent_dropout_mask = None

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 6),
                                        name='kernel',
                                        initializer=self.kernel_initializer,
                                        regularizer=self.kernel_regularizer,
                                        constraint=self.kernel_constraint)
        self.recurrent_kernel = self.add_weight(
                                        shape=(self.units, self.units * 5),
                                        name='recurrent_kernel',
                                        initializer=self.recurrent_initializer,
                                        regularizer=self.recurrent_regularizer,
                                        constraint=self.recurrent_constraint)
        self.NALU = self.add_weight(shape=(self.units * 4, self.units * 1),
                                        name='NALU',
                                        initializer=self.NALU_initializer,
                                        regularizer=self.NALU_regularizer,
                                        constraint=self.NALU_constraint)
        
        if self.use_bias:
            if self.unit_forget_bias:
                def bias_initializer(_, *args, **kwargs):
                    return K.concatenate([
                        self.bias_initializer((self.units,), *args, **kwargs),
                        initializers.Ones()((self.units,), *args, **kwargs),
                        self.bias_initializer((self.units * 4,), *args, **kwargs),
                    ])
            else:
                bias_initializer = self.bias_initializer
            self.bias = self.add_weight(shape=(self.units * 6,),
                                        name='bias',
                                        initializer=bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.kernel_i = self.kernel[:, self.units * 0: self.units * 1]
        self.kernel_f = self.kernel[:, self.units * 1: self.units * 2]
        self.kernel_c = self.kernel[:, self.units * 2: self.units * 3]
        self.kernel_o = self.kernel[:, self.units * 3: self.units * 4]
        self.kernel_b = self.kernel[:, self.units * 4: self.units * 5]
        self.kernel_g = self.kernel[:, self.units * 5: self.units * 6]

        self.recurrent_kernel_i = self.recurrent_kernel[:, self.units * 0: self.units * 1]
        self.recurrent_kernel_f = self.recurrent_kernel[:, self.units * 1: self.units * 2]
        self.recurrent_kernel_c = self.recurrent_kernel[:, self.units * 2: self.units * 3]
        self.recurrent_kernel_o = self.recurrent_kernel[:, self.units * 3: self.units * 4]
        self.recurrent_kernel_g = self.recurrent_kernel[:, self.units * 4: self.units * 5]

        self.NALU_M = self.NALU[self.units * 0: self.units * 2, :]
        self.NALU_W = self.NALU[self.units * 2: self.units * 4, :]
        
        if self.use_bias:
            self.bias_i = self.bias[self.units * 0: self.units * 1]
            self.bias_f = self.bias[self.units * 1: self.units * 2]
            self.bias_c = self.bias[self.units * 2: self.units * 3]
            self.bias_o = self.bias[self.units * 3: self.units * 4]
            self.bias_b = self.bias[self.units * 4: self.units * 5]
            self.bias_g = self.bias[self.units * 5: self.units * 6]
        else:
            self.bias_i = None
            self.bias_f = None
            self.bias_c = None
            self.bias_o = None
            self.bias_b = None
            self.bias_g = None
        self.built = True

    def call(self, inputs, states, training=None):
        if 0 < self.dropout < 1 and self._dropout_mask is None:
            self._dropout_mask = _generate_dropout_mask(
                K.ones_like(inputs),
                self.dropout,
                training=training,
                count=6)
        if (0 < self.recurrent_dropout < 1 and
                self._recurrent_dropout_mask is None):
            self._recurrent_dropout_mask = _generate_dropout_mask(
                K.ones_like(states[0]),
                self.recurrent_dropout,
                training=training,
                count=5)

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_mask = self._recurrent_dropout_mask

        h_tm1 = states[0]  # previous memory state
        c_tm1 = states[1]  # previous carry state


        if 0 < self.dropout < 1.:
            inputs_i = inputs * dp_mask[0]
            inputs_f = inputs * dp_mask[1]
            inputs_c = inputs * dp_mask[2]
            inputs_o = inputs * dp_mask[3]
            inputs_b = inputs * dp_mask[4]
            inputs_g = inputs * dp_mask[5]
        else:
            inputs_i = inputs
            inputs_f = inputs
            inputs_c = inputs
            inputs_o = inputs
            inputs_b = inputs
            inputs_g = inputs
        x_i = K.dot(inputs_i, self.kernel_i)
        x_f = K.dot(inputs_f, self.kernel_f)
        x_c = K.dot(inputs_c, self.kernel_c)
        x_o = K.dot(inputs_o, self.kernel_o)
        x_b = K.dot(inputs_b, self.kernel_b)
        x_g = K.dot(inputs_g, self.kernel_g)
        if self.use_bias:
            x_i = K.bias_add(x_i, self.bias_i)
            x_f = K.bias_add(x_f, self.bias_f)
            x_c = K.bias_add(x_c, self.bias_c)
            x_o = K.bias_add(x_o, self.bias_o)
            x_b = K.bias_add(x_b, self.bias_b)
            x_g = K.bias_add(x_g, self.bias_g)

        if 0 < self.recurrent_dropout < 1.:
            h_tm1_i = h_tm1 * rec_dp_mask[0]
            h_tm1_f = h_tm1 * rec_dp_mask[1]
            h_tm1_c = h_tm1 * rec_dp_mask[2]
            h_tm1_o = h_tm1 * rec_dp_mask[3]
            h_tm1_g = h_tm1 * rec_dp_mask[4]
        else:
            h_tm1_i = h_tm1
            h_tm1_f = h_tm1
            h_tm1_c = h_tm1
            h_tm1_o = h_tm1
            h_tm1_g = h_tm1

        h_i = K.dot(h_tm1_i, self.recurrent_kernel_i)
        h_f = K.dot(h_tm1_f, self.recurrent_kernel_f)
        h_c = K.dot(h_tm1_c, self.recurrent_kernel_c)
        h_o = K.dot(h_tm1_o, self.recurrent_kernel_o)
        h_g = K.dot(h_tm1_g, self.recurrent_kernel_g)

        
        f = K.hard_sigmoid(x_f + h_f)
        i = K.hard_sigmoid(x_i + h_i)
        c_hat = K.tanh(x_c + h_c)
        c_b = self.activation(x_b) * c_hat # the input is forwarded through this variable to 'c'
        
        MW = K.sigmoid(self.NALU_M) * K.sigmoid(self.NALU_W)
        conc = concatenate([f * c_tm1, i * c_b])
        #c_a = K.dot(conc, MW)
        c_m = tf.cast(tf.exp(tf.matmul(tf.log(tf.cast(conc + 1e-8, tf.complex64) ), tf.cast(MW, tf.complex64) )), tf.float32)
        #c_m = K.exp(K.dot(K.log(K.abs(conc) + 1e-8), MW))
        
        c_a = f * c_tm1 + i * c_b # 'add'
        #c_m = f * c_tm1 * i * c_b # 'multiply'
        
        g = K.sigmoid(x_g + h_g) # this will decide whether 'add' or 'multiply'
        c = g * c_a + (1 - g) * c_m
        #c = 0 * c_a + 1 * c_m
        o = K.hard_sigmoid(x_o + h_o)


        h = o * self.activation(c)
        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True
        return h, [h, c]

    def get_config(self):
        config = {'units': self.units,
                  'activation': activations.serialize(self.activation),
                  'recurrent_activation':
                      activations.serialize(self.recurrent_activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer':
                      initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer':
                      initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'unit_forget_bias': self.unit_forget_bias,
                  'kernel_regularizer':
                      regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer':
                      regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint':
                      constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(LSTM2Cell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



class LSTM2(RNN):
    """Long Short-Term Memory layer - Hochreiter 1997.
    # Arguments
        units: Positive integer, dimensionality of the output space.
        activation: Activation function to use
            (see [activations](../activations.md)).
            Default: hyperbolic tangent (`tanh`).
            If you pass `None`, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        recurrent_activation: Activation function to use
            for the recurrent step
            (see [activations](../activations.md)).
            Default: hard sigmoid (`hard_sigmoid`).
            If you pass `None`, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs.
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state.
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al. (2015)](
            http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf).
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, either 1 or 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
        return_sequences: Boolean. Whether to return the last output
            in the output sequence, or the full sequence.
        return_state: Boolean. Whether to return the last state
            in addition to the output.
        go_backwards: Boolean (default False).
            If True, process the input sequence backwards and return the
            reversed sequence.
        stateful: Boolean (default False). If True, the last state
            for each sample at index i in a batch will be used as initial
            state for the sample of index i in the following batch.
        unroll: Boolean (default False).
            If True, the network will be unrolled,
            else a symbolic loop will be used.
            Unrolling can speed-up a RNN,
            although it tends to be more memory-intensive.
            Unrolling is only suitable for short sequences.
    # References
        - [Long short-term memory](
          http://www.bioinf.jku.at/publications/older/2604.pdf)
        - [Learning to forget: Continual prediction with LSTM](
          http://www.mitpressjournals.org/doi/pdf/10.1162/089976600300015015)
        - [Supervised sequence labeling with recurrent neural networks](
          http://www.cs.toronto.edu/~graves/preprint.pdf)
        - [A Theoretically Grounded Application of Dropout in
           Recurrent Neural Networks](https://arxiv.org/abs/1512.05287)
    """

    def __init__(self, units,
                 activation='tanh',
                 recurrent_activation='hard_sigmoid',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 unit_forget_bias=True,
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=1,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if implementation == 0:
            warnings.warn('`implementation=0` has been deprecated, '
                          'and now defaults to `implementation=1`.'
                          'Please update your layer call.')
        if K.backend() == 'theano' and (dropout or recurrent_dropout):
            warnings.warn(
                'RNN dropout is no longer supported with the Theano backend '
                'due to technical limitations. '
                'You can either set `dropout` and `recurrent_dropout` to 0, '
                'or use the TensorFlow backend.')
            dropout = 0.
            recurrent_dropout = 0.

        cell = LSTM2Cell(units,
                        activation=activation,
                        recurrent_activation=recurrent_activation,
                        use_bias=use_bias,
                        kernel_initializer=kernel_initializer,
                        recurrent_initializer=recurrent_initializer,
                        unit_forget_bias=unit_forget_bias,
                        bias_initializer=bias_initializer,
                        kernel_regularizer=kernel_regularizer,
                        recurrent_regularizer=recurrent_regularizer,
                        bias_regularizer=bias_regularizer,
                        kernel_constraint=kernel_constraint,
                        recurrent_constraint=recurrent_constraint,
                        bias_constraint=bias_constraint,
                        dropout=dropout,
                        recurrent_dropout=recurrent_dropout,
                        implementation=implementation)
        super(LSTM2, self).__init__(cell,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   go_backwards=go_backwards,
                                   stateful=stateful,
                                   unroll=unroll,
                                   **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

    def call(self, inputs, mask=None, training=None, initial_state=None):
        self.cell._dropout_mask = None
        self.cell._recurrent_dropout_mask = None
        return super(LSTM2, self).call(inputs,
                                      mask=mask,
                                      training=training,
                                      initial_state=initial_state)

    @property
    def units(self):
        return self.cell.units

    @property
    def activation(self):
        return self.cell.activation

    @property
    def recurrent_activation(self):
        return self.cell.recurrent_activation

    @property
    def use_bias(self):
        return self.cell.use_bias

    @property
    def kernel_initializer(self):
        return self.cell.kernel_initializer

    @property
    def recurrent_initializer(self):
        return self.cell.recurrent_initializer

    @property
    def bias_initializer(self):
        return self.cell.bias_initializer

    @property
    def unit_forget_bias(self):
        return self.cell.unit_forget_bias

    @property
    def kernel_regularizer(self):
        return self.cell.kernel_regularizer

    @property
    def recurrent_regularizer(self):
        return self.cell.recurrent_regularizer

    @property
    def bias_regularizer(self):
        return self.cell.bias_regularizer

    @property
    def kernel_constraint(self):
        return self.cell.kernel_constraint

    @property
    def recurrent_constraint(self):
        return self.cell.recurrent_constraint

    @property
    def bias_constraint(self):
        return self.cell.bias_constraint

    @property
    def dropout(self):
        return self.cell.dropout

    @property
    def recurrent_dropout(self):
        return self.cell.recurrent_dropout

    @property
    def implementation(self):
        return self.cell.implementation

    def get_config(self):
        config = {'units': self.units,
                  'activation': activations.serialize(self.activation),
                  'recurrent_activation':
                      activations.serialize(self.recurrent_activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer':
                      initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer':
                      initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'unit_forget_bias': self.unit_forget_bias,
                  'kernel_regularizer':
                      regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer':
                      regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer':
                      regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint':
                      constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(LSTM2, self).get_config()
        del base_config['cell']
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        if 'implementation' in config and config['implementation'] == 0:
            config['implementation'] = 1
        return cls(**config)


def _generate_dropout_mask(ones, rate, training=None, count=1):
    def dropped_inputs():
        return K.dropout(ones, rate)

    if count > 1:
        return [K.in_train_phase(
            dropped_inputs,
            ones,
            training=training) for _ in range(count)]
    return K.in_train_phase(
        dropped_inputs,
        ones,
        training=training)


def _standardize_args(inputs, initial_state, constants, num_constants):
    """Standardize `__call__` to a single list of tensor inputs.
    When running a model loaded from file, the input tensors
    `initial_state` and `constants` can be passed to `RNN.__call__` as part
    of `inputs` instead of by the dedicated keyword arguments. This method
    makes sure the arguments are separated and that `initial_state` and
    `constants` are lists of tensors (or None).
    # Arguments
        inputs: tensor or list/tuple of tensors
        initial_state: tensor or list of tensors or None
        constants: tensor or list of tensors or None
    # Returns
        inputs: tensor
        initial_state: list of tensors or None
        constants: list of tensors or None
    """
    if isinstance(inputs, list):
        assert initial_state is None and constants is None
        if num_constants is not None:
            constants = inputs[-num_constants:]
            inputs = inputs[:-num_constants]
        if len(inputs) > 1:
            initial_state = inputs[1:]
        inputs = inputs[0]

    def to_list_or_none(x):
        if x is None or isinstance(x, list):
            return x
        if isinstance(x, tuple):
            return list(x)
        return [x]

    initial_state = to_list_or_none(initial_state)
    constants = to_list_or_none(constants)

    return inputs, initial_state, constants

In [71]:
import numpy as np
import keras.backend as K
from keras.layers import *
from keras.initializers import *
from keras.models import *
import tensorflow as tf
from keras.callbacks import Callback, EarlyStopping

class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)   

In [72]:
from keras.layers import LSTM, SimpleRNN, Embedding, Input, Dropout, Activation, TimeDistributed, Dense
from keras.optimizers import Adam, RMSprop, SGD
from keras.models import Model, Sequential

input_ = Input(shape=(max_seq_length,num_tokens+1), name='Input')
x = LSTM2(128, activation='softplus', recurrent_dropout=0.2, return_sequences=True, name='LSTM0')(input_)
x = LSTM2(128, activation='softplus', recurrent_dropout=0.2, return_sequences=False, name='LSTM1')(x)
out = Dense(1, activation='softplus', name='Dense')(x)

model = Model(input_, out)


epoch_size = train_x.shape[0]*1.0
batch_size = 2**3

schedule = SGDRScheduler(min_lr=1e-5,
                         max_lr=1e-2,
                         steps_per_epoch=np.ceil(epoch_size/batch_size),
                         lr_decay=1.0,
                         cycle_length=1,
                         mult_factor=1)

optimizer = Adam(lr=0.01)

model.compile(optimizer=optimizer, loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 31, 22)            0         
_________________________________________________________________
LSTM0 (LSTM2)                (None, 31, 128)           165120    
_________________________________________________________________
LSTM1 (LSTM2)                (None, 128)               246528    
_________________________________________________________________
Dense (Dense)                (None, 1)                 129       
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.fit(train_x, train_y, epochs=1000, verbose=1, validation_data=(valid_x, valid_y), batch_size=batch_size)

Train on 167 samples, validate on 833 samples
Epoch 1/1000
167/167 [==============================] - 17s 100ms/step - loss: 153422.8334 - val_loss: 90783.7219
Epoch 2/1000
167/167 [==============================] - 10s 63ms/step - loss: 94820.6448 - val_loss: 85301.6283
Epoch 3/1000
167/167 [==============================] - 10s 62ms/step - loss: 92493.3319 - val_loss: 101953.1583
Epoch 4/1000
167/167 [==============================] - 10s 63ms/step - loss: 90759.4140 - val_loss: 82829.0700
Epoch 5/1000
167/167 [==============================] - 10s 63ms/step - loss: 94792.3150 - val_loss: 96263.0741
Epoch 6/1000
167/167 [==============================] - 10s 61ms/step - loss: 84855.6663 - val_loss: 67233.9302
Epoch 7/1000
167/167 [==============================] - 10s 63ms/step - loss: 63593.3470 - val_loss: 42809.8707
Epoch 8/1000
167/167 [==============================] - 11s 65ms/step - loss: 63014.0630 - val_loss: 36600.0839
Epoch 9/1000
167/167 [==============================] -

167/167 [==============================] - 11s 66ms/step - loss: 579.6322 - val_loss: 781.7003
Epoch 75/1000
167/167 [==============================] - 10s 63ms/step - loss: 722.9220 - val_loss: 1248.1732
Epoch 76/1000
167/167 [==============================] - 11s 63ms/step - loss: 558.5313 - val_loss: 745.1189
Epoch 77/1000
167/167 [==============================] - 11s 64ms/step - loss: 738.9203 - val_loss: 2455.2329
Epoch 78/1000
167/167 [==============================] - 10s 62ms/step - loss: 941.2509 - val_loss: 871.5166
Epoch 79/1000
167/167 [==============================] - 10s 61ms/step - loss: 1166.1003 - val_loss: 1360.4407
Epoch 80/1000
167/167 [==============================] - 11s 64ms/step - loss: 701.4852 - val_loss: 907.1352
Epoch 81/1000
167/167 [==============================] - 11s 63ms/step - loss: 380.2055 - val_loss: 629.5037
Epoch 82/1000
167/167 [==============================] - 11s 65ms/step - loss: 515.6982 - val_loss: 514.4764
Epoch 83/1000
167/167 [======

Epoch 149/1000
167/167 [==============================] - 10s 63ms/step - loss: 203.4810 - val_loss: 339.0345
Epoch 150/1000
167/167 [==============================] - 10s 61ms/step - loss: 184.3444 - val_loss: 236.1781
Epoch 151/1000
167/167 [==============================] - 11s 63ms/step - loss: 146.9370 - val_loss: 293.6587
Epoch 152/1000
167/167 [==============================] - 11s 63ms/step - loss: 114.6121 - val_loss: 182.5353
Epoch 153/1000
167/167 [==============================] - 10s 62ms/step - loss: 139.5600 - val_loss: 148.1664
Epoch 154/1000
167/167 [==============================] - 11s 65ms/step - loss: 145.2523 - val_loss: 425.6265
Epoch 155/1000
167/167 [==============================] - 11s 64ms/step - loss: 245.2803 - val_loss: 168.1529
Epoch 156/1000
167/167 [==============================] - 10s 61ms/step - loss: 143.0080 - val_loss: 310.2952
Epoch 157/1000
167/167 [==============================] - 11s 63ms/step - loss: 226.6241 - val_loss: 240.0962
Epoch 158/

167/167 [==============================] - 11s 64ms/step - loss: 156.2684 - val_loss: 250.1036
Epoch 224/1000
167/167 [==============================] - 10s 62ms/step - loss: 190.1555 - val_loss: 627.3138
Epoch 225/1000
167/167 [==============================] - 10s 63ms/step - loss: 213.7814 - val_loss: 249.9629
Epoch 226/1000
167/167 [==============================] - 11s 66ms/step - loss: 114.2509 - val_loss: 289.4693
Epoch 227/1000
167/167 [==============================] - 11s 63ms/step - loss: 127.6746 - val_loss: 211.0181
Epoch 228/1000
167/167 [==============================] - 10s 62ms/step - loss: 131.0618 - val_loss: 161.6260
Epoch 229/1000
167/167 [==============================] - 11s 65ms/step - loss: 132.9003 - val_loss: 138.1954
Epoch 230/1000
167/167 [==============================] - 10s 63ms/step - loss: 156.1610 - val_loss: 114.6201
Epoch 231/1000
167/167 [==============================] - 10s 62ms/step - loss: 107.1829 - val_loss: 116.2726
Epoch 232/1000
167/167 [=

167/167 [==============================] - 10s 61ms/step - loss: 183.7899 - val_loss: 148.5994
Epoch 299/1000
167/167 [==============================] - 10s 61ms/step - loss: 155.0760 - val_loss: 70.5919
Epoch 300/1000
167/167 [==============================] - 11s 63ms/step - loss: 108.6363 - val_loss: 129.3120
Epoch 301/1000
167/167 [==============================] - 11s 63ms/step - loss: 108.0341 - val_loss: 64.1438
Epoch 302/1000
167/167 [==============================] - 11s 64ms/step - loss: 60.1510 - val_loss: 135.5305
Epoch 303/1000
167/167 [==============================] - 10s 62ms/step - loss: 65.4660 - val_loss: 137.6686
Epoch 304/1000
167/167 [==============================] - 10s 61ms/step - loss: 51.6024 - val_loss: 62.2074
Epoch 305/1000
167/167 [==============================] - 10s 62ms/step - loss: 74.7220 - val_loss: 80.5481
Epoch 306/1000
167/167 [==============================] - 10s 61ms/step - loss: 69.9704 - val_loss: 59.3073
Epoch 307/1000
167/167 [===========

...

167/167 [==============================] - 11s 65ms/step - loss: 32.9990 - val_loss: 40.9677
Epoch 676/1000
167/167 [==============================] - 10s 63ms/step - loss: 28.7814 - val_loss: 26.6582
Epoch 677/1000
167/167 [==============================] - 11s 63ms/step - loss: 28.8339 - val_loss: 39.0773
Epoch 678/1000
167/167 [==============================] - 11s 64ms/step - loss: 37.1322 - val_loss: 69.9174
Epoch 679/1000
167/167 [==============================] - 10s 63ms/step - loss: 79.5197 - val_loss: 70.2789
Epoch 680/1000
167/167 [==============================] - 11s 64ms/step - loss: 49.6317 - val_loss: 24.9812
Epoch 681/1000
167/167 [==============================] - 11s 64ms/step - loss: 24.6614 - val_loss: 23.3002
Epoch 682/1000
167/167 [==============================] - 11s 63ms/step - loss: 39.6876 - val_loss: 31.1337
Epoch 683/1000
167/167 [==============================] - 11s 64ms/step - loss: 26.8260 - val_loss: 23.3591
Epoch 684/1000
167/167 [===================

Epoch 751/1000
167/167 [==============================] - 11s 64ms/step - loss: 18.6674 - val_loss: 32.5237
Epoch 752/1000
167/167 [==============================] - 11s 64ms/step - loss: 19.1610 - val_loss: 21.3101
Epoch 753/1000
167/167 [==============================] - 10s 63ms/step - loss: 14.2469 - val_loss: 17.3629
Epoch 754/1000
167/167 [==============================] - 11s 64ms/step - loss: 13.7668 - val_loss: 25.2091
Epoch 755/1000
167/167 [==============================] - 11s 63ms/step - loss: 12.2229 - val_loss: 31.9575
Epoch 756/1000
167/167 [==============================] - 11s 63ms/step - loss: 18.4786 - val_loss: 65.0184
Epoch 757/1000
167/167 [==============================] - 11s 64ms/step - loss: 18.3930 - val_loss: 18.6288
Epoch 758/1000
167/167 [==============================] - 11s 66ms/step - loss: 12.0265 - val_loss: 18.5510
Epoch 759/1000
167/167 [==============================] - 11s 63ms/step - loss: 14.6599 - val_loss: 19.7402
Epoch 760/1000
167/167 [====

Epoch 827/1000
167/167 [==============================] - 11s 66ms/step - loss: 10.8357 - val_loss: 16.9460
Epoch 828/1000
167/167 [==============================] - 11s 65ms/step - loss: 21.1277 - val_loss: 14.3271
Epoch 829/1000
167/167 [==============================] - 11s 65ms/step - loss: 9.7694 - val_loss: 13.7721
Epoch 830/1000
167/167 [==============================] - 11s 64ms/step - loss: 14.2772 - val_loss: 25.2811
Epoch 831/1000
167/167 [==============================] - 10s 63ms/step - loss: 25.5371 - val_loss: 18.0192
Epoch 832/1000
167/167 [==============================] - 11s 65ms/step - loss: 16.9822 - val_loss: 17.9746
Epoch 833/1000
167/167 [==============================] - 11s 65ms/step - loss: 28.6409 - val_loss: 18.2285
Epoch 834/1000
167/167 [==============================] - 11s 65ms/step - loss: 31.9049 - val_loss: 25.4990
Epoch 835/1000
167/167 [==============================] - 11s 63ms/step - loss: 16.8758 - val_loss: 15.7704
Epoch 836/1000
167/167 [=====

167/167 [==============================] - 10s 63ms/step - loss: 17.5288 - val_loss: 20.4723
Epoch 903/1000
167/167 [==============================] - 11s 63ms/step - loss: 16.7018 - val_loss: 24.1944
Epoch 904/1000
167/167 [==============================] - 11s 64ms/step - loss: 13.1624 - val_loss: 30.3865
Epoch 905/1000
167/167 [==============================] - 10s 62ms/step - loss: 17.9296 - val_loss: 18.1877
Epoch 906/1000
167/167 [==============================] - 11s 65ms/step - loss: 26.8527 - val_loss: 19.5834
Epoch 907/1000
167/167 [==============================] - 11s 65ms/step - loss: 19.4729 - val_loss: 18.2548
Epoch 908/1000
167/167 [==============================] - 10s 61ms/step - loss: 16.8634 - val_loss: 18.2654
Epoch 909/1000
167/167 [==============================] - 11s 65ms/step - loss: 11.6736 - val_loss: 17.4398
Epoch 910/1000
167/167 [==============================] - 11s 64ms/step - loss: 17.5433 - val_loss: 24.2130
Epoch 911/1000
167/167 [===================

KeyboardInterrupt: 

In [81]:
t = 48
for k in range(1, max_seq_length):
    x = valid_x[t:t+1].copy()
    x[0][k:] = 0
    word = []
    for c in x[0]:
        word.append(reverse_input_char_index[np.argmax(c)])
    print(''.join(word), model.predict(x)[0][0])

s~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 5.2536645e-19
se~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 1.3416368e-18
sev~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 1.4318164e-15
seve~~~~~~~~~~~~~~~~~~~~~~~~~~~ 1.1794117e-12
seven~~~~~~~~~~~~~~~~~~~~~~~~~~ 0.008318301
sevent~~~~~~~~~~~~~~~~~~~~~~~~~ 7.1744804
seventy~~~~~~~~~~~~~~~~~~~~~~~~ 52.00066
seventy-~~~~~~~~~~~~~~~~~~~~~~~ 67.69003
seventy-e~~~~~~~~~~~~~~~~~~~~~~ 62.255558
seventy-ei~~~~~~~~~~~~~~~~~~~~~ 74.99532
seventy-eig~~~~~~~~~~~~~~~~~~~~ 73.3511
seventy-eigh~~~~~~~~~~~~~~~~~~~ 81.604866
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 89.02332
seventy-eight~~~~~~~~~~~~~~~~~~ 

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import RNN
from keras import *
from tensorflow.python.ops import *

class LSTM2Cell(Layer):
  """Cell class for the LSTM layer.

  Arguments:
      units: Positive integer, dimensionality of the output space.
      activation: Activation function to use.
          Default: hyperbolic tangent (`tanh`).
          If you pass `None`, no activation is applied
          (ie. "linear" activation: `a(x) = x`).
      recurrent_activation: Activation function to use
          for the recurrent step.
          Default: hard sigmoid (`hard_sigmoid`).
          If you pass `None`, no activation is applied
          (ie. "linear" activation: `a(x) = x`).x
      use_bias: Boolean, whether the layer uses a bias vector.
      kernel_initializer: Initializer for the `kernel` weights matrix,
          used for the linear transformation of the inputs.
      recurrent_initializer: Initializer for the `recurrent_kernel`
          weights matrix,
          used for the linear transformation of the recurrent state.
      bias_initializer: Initializer for the bias vector.
      unit_forget_bias: Boolean.
          If True, add 1 to the bias of the forget gate at initialization.
          Setting it to true will also force `bias_initializer="zeros"`.
          This is recommended in [Jozefowicz et
            al.](http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf)
      kernel_regularizer: Regularizer function applied to
          the `kernel` weights matrix.
      recurrent_regularizer: Regularizer function applied to
          the `recurrent_kernel` weights matrix.
      bias_regularizer: Regularizer function applied to the bias vector.
      kernel_constraint: Constraint function applied to
          the `kernel` weights matrix.
      recurrent_constraint: Constraint function applied to
          the `recurrent_kernel` weights matrix.
      bias_constraint: Constraint function applied to the bias vector.
      dropout: Float between 0 and 1.
          Fraction of the units to drop for
          the linear transformation of the inputs.
      recurrent_dropout: Float between 0 and 1.
          Fraction of the units to drop for
          the linear transformation of the recurrent state.
      implementation: Implementation mode, either 1 or 2.
          Mode 1 will structure its operations as a larger number of
          smaller dot products and additions, whereas mode 2 will
          batch them into fewer, larger operations. These modes will
          have different performance profiles on different hardware and
          for different applications.
  """

  def __init__(self,
               units,
               activation='tanh',
               recurrent_activation='hard_sigmoid',
               use_bias=True,
               kernel_initializer='glorot_uniform',
               recurrent_initializer='orthogonal',
               bias_initializer='zeros',
               unit_forget_bias=True,
               kernel_regularizer=None,
               recurrent_regularizer=None,
               bias_regularizer=None,
               kernel_constraint=None,
               recurrent_constraint=None,
               bias_constraint=None,
               dropout=0.,
               recurrent_dropout=0.,
               implementation=1,
               **kwargs):
    super(LSTM2Cell, self).__init__(**kwargs)
    self.units = units
    self.activation = activations.get(activation)
    self.recurrent_activation = activations.get(recurrent_activation)
    self.use_bias = use_bias

    self.kernel_initializer = initializers.get(kernel_initializer)
    self.recurrent_initializer = initializers.get(recurrent_initializer)
    self.bias_initializer = initializers.get(bias_initializer)
    self.unit_forget_bias = unit_forget_bias

    self.kernel_regularizer = regularizers.get(kernel_regularizer)
    self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
    self.bias_regularizer = regularizers.get(bias_regularizer)

    self.kernel_constraint = constraints.get(kernel_constraint)
    self.recurrent_constraint = constraints.get(recurrent_constraint)
    self.bias_constraint = constraints.get(bias_constraint)

    self.dropout = min(1., max(0., dropout))
    self.recurrent_dropout = min(1., max(0., recurrent_dropout))
    self.implementation = implementation
    self.state_size = (self.units, self.units)
    self.output_size = self.units
    self._dropout_mask = None
    self._recurrent_dropout_mask = None

  def build(self, input_shape):
    input_dim = input_shape[-1]
    self.kernel = self.add_weight(
        shape=(input_dim, self.units * 4),
        name='kernel',
        initializer=self.kernel_initializer,
        regularizer=self.kernel_regularizer,
        constraint=self.kernel_constraint)
    self.recurrent_kernel = self.add_weight(
        shape=(self.units, self.units * 4),
        name='recurrent_kernel',
        initializer=self.recurrent_initializer,
        regularizer=self.recurrent_regularizer,
        constraint=self.recurrent_constraint)

    if self.use_bias:
      if self.unit_forget_bias:

        def bias_initializer(_, *args, **kwargs):
          return K.concatenate([
              self.bias_initializer((self.units,), *args, **kwargs),
              initializers.Ones()((self.units,), *args, **kwargs),
              self.bias_initializer((self.units * 2,), *args, **kwargs),
          ])
      else:
        bias_initializer = self.bias_initializer
      self.bias = self.add_weight(
          shape=(self.units * 4,),
          name='bias',
          initializer=bias_initializer,
          regularizer=self.bias_regularizer,
          constraint=self.bias_constraint)
    else:
      self.bias = None
    self.built = True

  def call(self, inputs, states, training=None):
    if 0 < self.dropout < 1 and self._dropout_mask is None:
      self._dropout_mask = _generate_dropout_mask(
          array_ops.ones_like(inputs),
          self.dropout,
          training=training,
          count=4)
    if (0 < self.recurrent_dropout < 1 and
        self._recurrent_dropout_mask is None):
      self._recurrent_dropout_mask = _generate_dropout_mask(
          array_ops.ones_like(states[0]),
          self.recurrent_dropout,
          training=training,
          count=4)

    # dropout matrices for input units
    dp_mask = self._dropout_mask
    # dropout matrices for recurrent units
    rec_dp_mask = self._recurrent_dropout_mask

    h_tm1 = states[0]  # previous memory state
    c_tm1 = states[1]  # previous carry state

    if self.implementation == 1:
      if 0 < self.dropout < 1.:
        inputs_i = inputs * dp_mask[0]
        inputs_f = inputs * dp_mask[1]
        inputs_c = inputs * dp_mask[2]
        inputs_o = inputs * dp_mask[3]
      else:
        inputs_i = inputs
        inputs_f = inputs
        inputs_c = inputs
        inputs_o = inputs
      x_i = K.dot(inputs_i, self.kernel[:, :self.units])
      x_f = K.dot(inputs_f, self.kernel[:, self.units:self.units * 2])
      x_c = K.dot(inputs_c, self.kernel[:, self.units * 2:self.units * 3])
      x_o = K.dot(inputs_o, self.kernel[:, self.units * 3:])
      if self.use_bias:
        x_i = K.bias_add(x_i, self.bias[:self.units])
        x_f = K.bias_add(x_f, self.bias[self.units:self.units * 2])
        x_c = K.bias_add(x_c, self.bias[self.units * 2:self.units * 3])
        x_o = K.bias_add(x_o, self.bias[self.units * 3:])

      if 0 < self.recurrent_dropout < 1.:
        h_tm1_i = h_tm1 * rec_dp_mask[0]
        h_tm1_f = h_tm1 * rec_dp_mask[1]
        h_tm1_c = h_tm1 * rec_dp_mask[2]
        h_tm1_o = h_tm1 * rec_dp_mask[3]
      else:
        h_tm1_i = h_tm1
        h_tm1_f = h_tm1
        h_tm1_c = h_tm1
        h_tm1_o = h_tm1
      i = self.recurrent_activation(
          x_i + K.dot(h_tm1_i, self.recurrent_kernel[:, :self.units]))
      f = self.recurrent_activation(
          x_f + K.dot(h_tm1_f,
                      self.recurrent_kernel[:, self.units: self.units * 2]))
      # self.activation(x_i) *           is added to c, so it can store more info
      c = f * c_tm1 + self.activation(x_i) * i * K.tanh(
          x_c + K.dot(h_tm1_c,
                      self.recurrent_kernel[:, self.units * 2: self.units * 3]))
      o = self.recurrent_activation(
          x_o + K.dot(h_tm1_o, self.recurrent_kernel[:, self.units * 3:]))
    else:
      if 0. < self.dropout < 1.:
        inputs *= dp_mask[0]
      z = K.dot(inputs, self.kernel)
      if 0. < self.recurrent_dropout < 1.:
        h_tm1 *= rec_dp_mask[0]
      z += K.dot(h_tm1, self.recurrent_kernel)
      if self.use_bias:
        z = K.bias_add(z, self.bias)

      z0 = z[:, :self.units]
      z1 = z[:, self.units:2 * self.units]
      z2 = z[:, 2 * self.units:3 * self.units]
      z3 = z[:, 3 * self.units:]

      i = self.recurrent_activation(z0)
      f = self.recurrent_activation(z1)
      c = f * c_tm1 + i * K.tanh(z2)
      o = self.recurrent_activation(z3)

    h = o * self.activation(c)

    return h, [h, c]

  def get_config(self):
    config = {
        'units':
            self.units,
        'activation':
            activations.serialize(self.activation),
        'recurrent_activation':
            activations.serialize(self.recurrent_activation),
        'use_bias':
            self.use_bias,
        'kernel_initializer':
            initializers.serialize(self.kernel_initializer),
        'recurrent_initializer':
            initializers.serialize(self.recurrent_initializer),
        'bias_initializer':
            initializers.serialize(self.bias_initializer),
        'unit_forget_bias':
            self.unit_forget_bias,
        'kernel_regularizer':
            regularizers.serialize(self.kernel_regularizer),
        'recurrent_regularizer':
            regularizers.serialize(self.recurrent_regularizer),
        'bias_regularizer':
            regularizers.serialize(self.bias_regularizer),
        'kernel_constraint':
            constraints.serialize(self.kernel_constraint),
        'recurrent_constraint':
            constraints.serialize(self.recurrent_constraint),
        'bias_constraint':
            constraints.serialize(self.bias_constraint),
        'dropout':
            self.dropout,
        'recurrent_dropout':
            self.recurrent_dropout,
        'implementation':
            self.implementation
    }
    base_config = super(LSTM2Cell, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))



class LSTM2(RNN):
    """Long Short-Term Memory layer - Hochreiter 1997.
    # Arguments
        units: Positive integer, dimensionality of the output space.
        activation: Activation function to use
            (see [activations](../activations.md)).
            Default: hyperbolic tangent (`tanh`).
            If you pass `None`, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        recurrent_activation: Activation function to use
            for the recurrent step
            (see [activations](../activations.md)).
            Default: hard sigmoid (`hard_sigmoid`).
            If you pass `None`, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs.
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state.
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al. (2015)](
            http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf).
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, either 1 or 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
        return_sequences: Boolean. Whether to return the last output
            in the output sequence, or the full sequence.
        return_state: Boolean. Whether to return the last state
            in addition to the output.
        go_backwards: Boolean (default False).
            If True, process the input sequence backwards and return the
            reversed sequence.
        stateful: Boolean (default False). If True, the last state
            for each sample at index i in a batch will be used as initial
            state for the sample of index i in the following batch.
        unroll: Boolean (default False).
            If True, the network will be unrolled,
            else a symbolic loop will be used.
            Unrolling can speed-up a RNN,
            although it tends to be more memory-intensive.
            Unrolling is only suitable for short sequences.
    # References
        - [Long short-term memory](
          http://www.bioinf.jku.at/publications/older/2604.pdf)
        - [Learning to forget: Continual prediction with LSTM](
          http://www.mitpressjournals.org/doi/pdf/10.1162/089976600300015015)
        - [Supervised sequence labeling with recurrent neural networks](
          http://www.cs.toronto.edu/~graves/preprint.pdf)
        - [A Theoretically Grounded Application of Dropout in
           Recurrent Neural Networks](https://arxiv.org/abs/1512.05287)
    """

    def __init__(self, units,
                 activation='tanh',
                 recurrent_activation='hard_sigmoid',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 unit_forget_bias=True,
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=1,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if implementation == 0:
            warnings.warn('`implementation=0` has been deprecated, '
                          'and now defaults to `implementation=1`.'
                          'Please update your layer call.')
        if K.backend() == 'theano' and (dropout or recurrent_dropout):
            warnings.warn(
                'RNN dropout is no longer supported with the Theano backend '
                'due to technical limitations. '
                'You can either set `dropout` and `recurrent_dropout` to 0, '
                'or use the TensorFlow backend.')
            dropout = 0.
            recurrent_dropout = 0.

        cell = LSTM2Cell(units,
                        activation=activation,
                        recurrent_activation=recurrent_activation,
                        use_bias=use_bias,
                        kernel_initializer=kernel_initializer,
                        recurrent_initializer=recurrent_initializer,
                        unit_forget_bias=unit_forget_bias,
                        bias_initializer=bias_initializer,
                        kernel_regularizer=kernel_regularizer,
                        recurrent_regularizer=recurrent_regularizer,
                        bias_regularizer=bias_regularizer,
                        kernel_constraint=kernel_constraint,
                        recurrent_constraint=recurrent_constraint,
                        bias_constraint=bias_constraint,
                        dropout=dropout,
                        recurrent_dropout=recurrent_dropout,
                        implementation=implementation)
        super(LSTM2, self).__init__(cell,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   go_backwards=go_backwards,
                                   stateful=stateful,
                                   unroll=unroll,
                                   **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

    def call(self, inputs, mask=None, training=None, initial_state=None):
        self.cell._dropout_mask = None
        self.cell._recurrent_dropout_mask = None
        return super(LSTM2, self).call(inputs,
                                      mask=mask,
                                      training=training,
                                      initial_state=initial_state)

    @property
    def units(self):
        return self.cell.units

    @property
    def activation(self):
        return self.cell.activation

    @property
    def recurrent_activation(self):
        return self.cell.recurrent_activation

    @property
    def use_bias(self):
        return self.cell.use_bias

    @property
    def kernel_initializer(self):
        return self.cell.kernel_initializer

    @property
    def recurrent_initializer(self):
        return self.cell.recurrent_initializer

    @property
    def bias_initializer(self):
        return self.cell.bias_initializer

    @property
    def unit_forget_bias(self):
        return self.cell.unit_forget_bias

    @property
    def kernel_regularizer(self):
        return self.cell.kernel_regularizer

    @property
    def recurrent_regularizer(self):
        return self.cell.recurrent_regularizer

    @property
    def bias_regularizer(self):
        return self.cell.bias_regularizer

    @property
    def kernel_constraint(self):
        return self.cell.kernel_constraint

    @property
    def recurrent_constraint(self):
        return self.cell.recurrent_constraint

    @property
    def bias_constraint(self):
        return self.cell.bias_constraint

    @property
    def dropout(self):
        return self.cell.dropout

    @property
    def recurrent_dropout(self):
        return self.cell.recurrent_dropout

    @property
    def implementation(self):
        return self.cell.implementation

    def get_config(self):
        config = {'units': self.units,
                  'activation': activations.serialize(self.activation),
                  'recurrent_activation':
                      activations.serialize(self.recurrent_activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer':
                      initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer':
                      initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'unit_forget_bias': self.unit_forget_bias,
                  'kernel_regularizer':
                      regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer':
                      regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer':
                      regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint':
                      constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(LSTM2, self).get_config()
        del base_config['cell']
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        if 'implementation' in config and config['implementation'] == 0:
            config['implementation'] = 1
        return cls(**config)


def _generate_dropout_mask(ones, rate, training=None, count=1):
    def dropped_inputs():
        return K.dropout(ones, rate)

    if count > 1:
        return [K.in_train_phase(
            dropped_inputs,
            ones,
            training=training) for _ in range(count)]
    return K.in_train_phase(
        dropped_inputs,
        ones,
        training=training)


def _standardize_args(inputs, initial_state, constants, num_constants):
    """Standardize `__call__` to a single list of tensor inputs.
    When running a model loaded from file, the input tensors
    `initial_state` and `constants` can be passed to `RNN.__call__` as part
    of `inputs` instead of by the dedicated keyword arguments. This method
    makes sure the arguments are separated and that `initial_state` and
    `constants` are lists of tensors (or None).
    # Arguments
        inputs: tensor or list/tuple of tensors
        initial_state: tensor or list of tensors or None
        constants: tensor or list of tensors or None
    # Returns
        inputs: tensor
        initial_state: list of tensors or None
        constants: list of tensors or None
    """
    if isinstance(inputs, list):
        assert initial_state is None and constants is None
        if num_constants is not None:
            constants = inputs[-num_constants:]
            inputs = inputs[:-num_constants]
        if len(inputs) > 1:
            initial_state = inputs[1:]
        inputs = inputs[0]

    def to_list_or_none(x):
        if x is None or isinstance(x, list):
            return x
        if isinstance(x, tuple):
            return list(x)
        return [x]

    initial_state = to_list_or_none(initial_state)
    constants = to_list_or_none(constants)

    return inputs, initial_state, constants